# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import pprint
%matplotlib inline

import os
print(os.listdir("../input")) #loading all the files on input folder

Using TensorFlow backend.
C:\Users\Shanaka-PC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Shanaka-PC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Shanaka-PC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Shanaka-PC\anaconda3\lib\site-packages\tensorflow\python

['Colo - Temp.csv', 'Colo-Humidity.csv', 'New folder', 'precentage error calcuate.png', 'PredictedTemp.csv', 'PredictedTemp2019.csv', 'Temperature_(2020).html', 'Temperature_(2020).ipynb', 'Temp_(2020)', 'Temp_(2020).h5']


In [2]:
dataframe = pd.read_csv('../input/Colo-Humidity.csv') #adding 'Colo-Humidity' csv to the dataframe

In [3]:
dataframe #displaying the dataframe

,Date,Humidity
0,1/1/2009,62
1,2/1/2009,59
2,3/1/2009,58
3,4/1/2009,66
4,5/1/2009,68
...,...,...
4012,27/12/2019,77
4013,28/12/2019,78
4014,29/12/2019,76
4015,30/12/2019,78


In [4]:
dataframe.Date=pd.to_datetime(dataframe.Date)
dataframe=dataframe.set_index('Date') #setting the index by the date

In [5]:
dataframe #displaying the dataframe

,Humidity
Date,
2009-01-01,62
2009-02-01,59
2009-03-01,58
2009-04-01,66
2009-05-01,68
...,...
2019-12-27,77
2019-12-28,78
2019-12-29,76


In [6]:
TestData = dataframe.tail(365) #setting the testing set as the last year of the dataset

In [7]:
DfTotal = pd.concat((dataframe[["Humidity"]], TestData[["Humidity"]]), axis=0) #concating the dataset with the training set

In [8]:
DfTotal.shape #getting the shape 

(4382, 1)

In [9]:
inputs = DfTotal[len(DfTotal) - len(TestData) - 60:].values
inputs.shape

(425, 1)

# Generating new dates

In [10]:
from pandas.tseries.offsets import DateOffset
AddDates = [dataframe.index[-1] + DateOffset(days=x) for x in range(0,366)]
FutureDates = pd.DataFrame(index=AddDates[1:],columns=dataframe.columns)

In [11]:
FutureDates.tail(365)

,Humidity
2020-01-01,NaN
2020-01-02,NaN
2020-01-03,NaN
2020-01-04,NaN
2020-01-05,NaN
...,...
2020-12-26,NaN
2020-12-27,NaN
2020-12-28,NaN
2020-12-29,NaN


In [12]:
dataframe.shape

(4017, 1)

In [13]:
TrainingSet = dataframe #passing the full dataset for training

In [14]:
TrainingSet = TrainingSet.values
sc = MinMaxScaler(feature_range=(0, 1))
Train = sc.fit_transform(TrainingSet)

*Setting Min Max Scaler with the default feature range to the training set 

In [15]:
Train.shape

(4017, 1)

In [16]:
X_Train = []
Y_Train = []

for i in range(60, Train.shape[0]): # Range should be from 60 Values to END
   
    X_Train.append(Train[i-60:i]) # X_Train 0-59
   
    Y_Train.append(Train[i]) # Y Would be 60 th Value based on past 60 Values

# Convert into Numpy Array
X_Train = np.array(X_Train)
Y_Train = np.array(Y_Train)

print(X_Train.shape)
print(Y_Train.shape)

(3957, 60, 1)
(3957, 1)


In [17]:
X_Train = np.reshape(X_Train, newshape=(X_Train.shape[0], X_Train.shape[1], 1))
X_Train.shape

(3957, 60, 1)

# Model

In [18]:
XploreReg = Sequential()

# Adding first LSTM layer with Dropout 
XploreReg.add(LSTM(units = 150, return_sequences = True,input_shape = (X_Train.shape[1], 1)))
XploreReg.add(Dropout(0.2))

# Adding second LSTM layer with Dropout
XploreReg.add(LSTM(units = 150, return_sequences = True))
XploreReg.add(Dropout(0.2))

# Adding third LSTM layer with Dropout
XploreReg.add(LSTM(units = 150, return_sequences = True))
XploreReg.add(Dropout(0.2))

# Adding fourth LSTM layer with Dropout
XploreReg.add(LSTM(units = 150))
XploreReg.add(Dropout(0.2))

# Adding the output layer
XploreReg.add(Dense(units = 1))

In [19]:
XploreReg.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 60, 150)           91200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 150)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 150)           180600    
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 150)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 60, 150)           180600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 60, 150)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 150)              

# Compiling the model

In [20]:
XploreReg.compile(optimizer = 'adam', loss = 'mean_squared_error')
XploreReg.fit(X_Train,Y_Train, epochs = 100, batch_size = 6)


Epoch 1/100
3957/3957 [==============================] - 94s 24ms/step - loss: 0.0185
Epoch 2/100
3957/3957 [==============================] - 105s 26ms/step - loss: 0.0127
Epoch 3/100
3957/3957 [==============================] - 117s 30ms/step - loss: 0.0102
Epoch 4/100
3957/3957 [==============================] - 120s 30ms/step - loss: 0.0083
Epoch 5/100
3957/3957 [==============================] - 105s 27ms/step - loss: 0.0078
Epoch 6/100
3957/3957 [==============================] - 106s 27ms/step - loss: 0.0072
Epoch 7/100
3957/3957 [==============================] - 106s 27ms/step - loss: 0.0072
Epoch 8/100
3957/3957 [==============================] - 106s 27ms/step - loss: 0.0070
Epoch 9/100
3957/3957 [==============================] - 107s 27ms/step - loss: 0.0072
Epoch 10/100
3957/3957 [==============================] - 107s 27ms/step - loss: 0.0070
Epoch 11/100
3957/3957 [==============================] - 115s 29ms/step - loss: 0.0068
Epoch 12/100
3957/3957 [=================

3957/3957 [==============================] - 106s 27ms/step - loss: 0.0053
Epoch 92/100
3957/3957 [==============================] - 106s 27ms/step - loss: 0.0052
Epoch 93/100
3957/3957 [==============================] - 106s 27ms/step - loss: 0.0053
Epoch 94/100
3957/3957 [==============================] - 106s 27ms/step - loss: 0.0052
Epoch 95/100
3957/3957 [==============================] - 105s 27ms/step - loss: 0.0051
Epoch 96/100
3957/3957 [==============================] - 107s 27ms/step - loss: 0.0052
Epoch 97/100
3957/3957 [==============================] - 106s 27ms/step - loss: 0.0051
Epoch 98/100
3957/3957 [==============================] - 105s 27ms/step - loss: 0.0051
Epoch 99/100
3957/3957 [==============================] - 106s 27ms/step - loss: 0.0051
Epoch 100/100
3957/3957 [==============================] - 106s 27ms/step - loss: 0.0050


Per Epoch Time for CPU based 
1. i5 8th gen laptop = 250s/4mins
2. i7 8th gen laptop = 150s/2.5mins
3. i3 8th gen laptop = 110s/1min


In [21]:
FutureDates.shape

(365, 1)

In [22]:
inputs = DfTotal[len(DfTotal) - len(TestData) - 60:].values

# We need to Reshape
inputs = inputs.reshape(-1,1)

# Normalize the Dataset
inputs = sc.transform(inputs)

X_test = []
for i in range(60, 425):
    X_test.append(inputs[i-60:i])
       
# Convert into Numpy Array
X_test = np.array(X_test)

# Reshape before Passing to Network
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Pass to Model
predicted_Humid = XploreReg.predict(X_test)

# Do inverse Transformation to get Values
predicted_Humid = sc.inverse_transform(predicted_Humid)

In [23]:
True_Humid = 0
Predicted_Humid  = predicted_Humid
dates = TestData.index.to_list()

In [24]:
Predicted_Df = pd.DataFrame(data={
    "Date":FutureDates.index.to_list(),
    "TrueHumidity": True_Humid,
    "PredictedHumidity":[x[0] for x in predicted_Humid]
})

In [25]:
Predicted_Df

,Date,TrueHumidity,PredictedHumidity
0,2020-01-01,0,80.027321
1,2020-01-02,0,76.054855
2,2020-01-03,0,75.042053
3,2020-01-04,0,74.646797
4,2020-01-05,0,75.091187
...,...,...,...
360,2020-12-26,0,80.694260
361,2020-12-27,0,77.007370
362,2020-12-28,0,79.322449
363,2020-12-29,0,77.041809


In [26]:
Predicted_Df.to_csv("../input/Humidity.csv")

In [27]:
XploreReg.save('../input/Humidity_(2020)') 

In [28]:
XploreReg.save('../input/Humidity_(2020).h5') 